In [1]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.utils import to_categorical
import numpy as np
import math
import os
import pandas as pd

check = True 


# load data 

# 개인 아이디어 : 주장을 넣었을 때, ChatGPT 에 근거를 넣기
# 현재 모델은 어떤 단어가 들어갔는지만 고려. 깊게 발전시키지 않음. 

# 데이터를 보니, 의견을 알리는 것도 있음. 즉, 사실 주장을 증명하기엔 애매모호한 주제들이 많음. 
# 데이터셋도 필터링이 필요할 듯. 흐으음.. 한편으론 글의 타입이 변하기만 해도 애매한 부분이


# load tokenize 
# make model & train 
# 학습한 모델 불러오기 -> 모델이 상당히 조잡한데... 


csv_location = f"C:/Users/user/Desktop/test_ethic/20231110_total.csv"


def to_one_hot(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

df = pd.read_csv(csv_location, encoding='CP949')

def embedding(data, max_words, word_index = None) : 
    tokenizer = Tokenizer(num_words=max_words)
    # Tokenizer 내부의 단어-인덱스 매핑을 구축함. 
    tokenizer.fit_on_texts(data)

    # 단어 - 인덱스 매핑 저장하기 
    if word_index is None : 
        new_word_index = tokenizer.word_index
    else : 
        tokenizer.word_index = word_index
    

    # texts 에 담겨있는 텍스트 데이터를 정수 시퀀스로 변환
    train_data = tokenizer.texts_to_sequences(data) 
    if word_index is None : 
        return new_word_index, train_data 
    else : 
        return word_index, train_data



In [3]:
#def load_data(csv_location) : 
check = True 
if check == True : 
    df = pd.read_csv(csv_location, encoding='CP949')
    #df['label'] = df['label'].replace({'판단 유보' : -1, '사실' : 2, '대체로 사실': 2, '대체로 사실 아님' : 0, '전혀 사실 아님' : 0, '논쟁 중' : -1, '절반의 사실' : 1, '판단 유보' : 1})
    df['label'] = df['label'].replace({'판단 유보' : 1, '사실' : 2, '대체로 사실': 2, '대체로 사실 아님' : 0, '전혀 사실 아님' : 0, '논쟁 중' : 1, '절반의 사실' : 1, '판단 유보' : 1})


    filtered_df = df[df['label'] != -1]

    train_texts = filtered_df["제목"].tolist()

    train_label = filtered_df['label'].tolist()
    
    train_label = np.asarray(train_label).astype('float32')
    test_ratio = math.floor(len(train_texts) * 0.2)

    max_words = 1000
    maxlen = 1000

    word_index, train_data = embedding(data = train_texts, max_words=max_words)    
    ont_hot_train_data = to_one_hot(train_data, dimension=max_words)
    
    x_train = ont_hot_train_data[test_ratio:]
    y_train = train_label[test_ratio:]
    y_train_one_hot = to_categorical(y_train, num_classes=3)


    x_val = ont_hot_train_data[:test_ratio]
    y_val = train_label[:test_ratio]
    y_val_one_hot = to_categorical(y_val, num_classes=3)

In [4]:
filtered_df

,제목,출처,label,링크
0,길거리에서 자신의 가슴을 만져달라는 퍼포먼스는 공연음란죄로 처벌된다,출처: 인터넷 커뮤니티,1,https://bbs.ruliweb.com/community/board/300143...
1,우리나라 공공의료 비중은 OECD 국가 중 꼴찌다,"출처: 국립중앙의료원 이전신축 축소, 국감서도 관심…""병상 확보돼야""",2,https://www.medipana.com/article/view.php?news...
2,"빈대 방역 위해 뿌리는 규조토 분말, 인체에 해롭지 않다","출처: 인터넷 커뮤니티 게시물(루리웹, 2023년 11월 3일)",0,https://bbs.ruliweb.com/community/board/300148...
3,우리나라는 허위정보 발언에 대한 국회의원 면책특권 범위가 해외보다 넓다,출처: 조선일보 [사설] 가짜 뉴스로 사익까지 챙길 수 있는 한국 의원의 특권(20...,2,https://www.chosun.com/opinion/editorial/2023/...
4,수도권 제4매립지가 김포 땅이어서 쓰레기 문제를 해결할 수 있다,출처: 매일경제 김병수 김포시장 인터뷰,0,https://www.mk.co.kr/news/society/10849275
...,...,...,...,...
4644,"""더불어민주당도 국모닝 했다""",출처: '썰전' 안철수 국민의당은 '문모닝'당? 정책발표는 기사가 안나와,1,http://www.mydaily.co.kr/new_yk/html/read.php?...
4645,“77%로 미군 주둔비 부담 커…일본은 50%”,"출처: 이재명 시장, ""주한 미군 주둔비 적정선에서 조정 필요""",0,http://news.joins.com/article/21073540
4646,현재 지지율 1위 문재인 민주당 경선 후보 아들과 관련한 취업특혜 논란이 있다.,출처:,1,url not available
4647,‘검수완박’ 추진은 헌법에 위반된다,출처: 관련보도(한국일보),1,https://www.hankookilbo.com/News/Read/A2022041...


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.optimizers import Adam


import numpy as np

if check == True : 

    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=50, input_length=maxlen))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(3, activation='softmax'))

    custom_optimizer = Adam(learning_rate=0.01)
    model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    history = model.fit(x_train, y_train_one_hot, epochs=50, batch_size=128,  validation_data=(x_val, y_val_one_hot))
    
    model.save('test.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 50)          50000     
                                                                 
 flatten_1 (Flatten)         (None, 50000)             0         
                                                                 
 dense_2 (Dense)             (None, 64)                3200064   
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 3,250,259
Trainable params: 3,250,259
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
30/30 [==============================] - 2s 67ms/step - loss: 2.6946 - accuracy: 0.5245 - val_loss: 1.0763 - val_accuracy: 0.4865
Epoch 2/50
30/30 [=================

In [27]:
from tensorflow.keras.models import load_model
import pandas as pd
import random 

# 모델 로드
model = load_model('test.h5')

test_df = pd.read_csv(csv_location, encoding='CP949')
test_data = random.sample(test_df['제목'].tolist(), 100)

word_index, embed_data = embedding(data = test_data, max_words=max_words, word_index=word_index)    
ont_hot_train_data = to_one_hot(embed_data, dimension=max_words)


# 새로운 테스트 데이터셋에 대한 예측
predictions = model.predict(ont_hot_train_data)  # new_x_test는 새로운 테스트 데이터셋
max_indices = np.argmax(predictions, axis=1)

# x_val 부분은 사전 전처리 코드를 추가해둬야겠네. 
# embedding 전의 데이터와 구분할 필요가 있겠다. 

#temporary_category = ['조선일보' for index in max_indices]
categories = ['JTBC', 'KBS', 'MBC', 'SBS']
temporary_category = [random.choice(categories) for _ in max_indices]

# 예측 결과와 테스트 데이터를 DataFrame으로 만들기

label_values = ['거짓', '논란', '사실']  # 실제 레이블에 맞게 수정
mapped_labels = [label_values[prediction] for prediction in max_indices]


result_df = pd.DataFrame({'test_data': test_data, 'Category' : temporary_category ,'prediction': mapped_labels})




4/4 [==============================] - 0s 2ms/step


In [28]:

temporary_category

['JTBC',
 'JTBC',
 'SBS',
 'MBC',
 'JTBC',
 'MBC',
 'JTBC',
 'KBS',
 'MBC',
 'KBS',
 'SBS',
 'SBS',
 'KBS',
 'MBC',
 'KBS',
 'MBC',
 'SBS',
 'JTBC',
 'MBC',
 'MBC',
 'SBS',
 'SBS',
 'SBS',
 'SBS',
 'JTBC',
 'JTBC',
 'MBC',
 'MBC',
 'JTBC',
 'JTBC',
 'MBC',
 'KBS',
 'KBS',
 'KBS',
 'JTBC',
 'KBS',
 'KBS',
 'JTBC',
 'MBC',
 'KBS',
 'SBS',
 'MBC',
 'JTBC',
 'SBS',
 'KBS',
 'JTBC',
 'MBC',
 'MBC',
 'MBC',
 'SBS',
 'KBS',
 'MBC',
 'KBS',
 'JTBC',
 'JTBC',
 'SBS',
 'KBS',
 'MBC',
 'KBS',
 'SBS',
 'SBS',
 'MBC',
 'MBC',
 'SBS',
 'KBS',
 'KBS',
 'KBS',
 'JTBC',
 'JTBC',
 'SBS',
 'SBS',
 'MBC',
 'KBS',
 'MBC',
 'SBS',
 'SBS',
 'SBS',
 'KBS',
 'JTBC',
 'KBS',
 'JTBC',
 'SBS',
 'SBS',
 'JTBC',
 'KBS',
 'JTBC',
 'JTBC',
 'MBC',
 'MBC',
 'MBC',
 'KBS',
 'KBS',
 'SBS',
 'SBS',
 'KBS',
 'SBS',
 'SBS',
 'SBS',
 'KBS',
 'MBC']

In [ ]:
# Softmax 함수를 사용하여 확률 분포 얻기
probabilities = model.predict(ont_hot_train_data, verbose=0)

# 각 예측에서 가장 높은 확률을 가진 클래스의 인덱스 가져오기
predicted_labels = np.argmax(probabilities, axis=1)

# 예측된 레이블을 실제 레이블 값으로 변환 (예: 0부터 시작하는 클래스 레이블)
# 여기서는 가정적으로 label_values를 정의하여 사용합니다. 실제로는 데이터셋에 따라 다르게 설정될 수 있습니다.


In [38]:
result_df.to_csv(f"total_result.csv", encoding='utf-8-sig', index=False)

In [36]:
category = list(set(result_df['Category'].tolist()))

info = []
for c in category : 
    filtered_df = result_df[result_df['Category'] == c]
    num_data = len(filtered_df)
    num_fact = len(filtered_df[filtered_df['prediction'] == '사실'])
    num_conv = len(filtered_df[filtered_df['prediction'] == '논란'])
    num_fake = len(filtered_df[filtered_df['prediction'] == '거짓'])

    info.append([c, num_data, num_fact, num_conv, num_fake])


columns = ['Category', 'Num_Data', 'Num_Fact', 'Num_Conv', 'Num_Fake']
df_info = pd.DataFrame(info, columns=columns)

print(df_info)


  Category  Num_Data  Num_Fact  Num_Conv  Num_Fake
0      SBS        27         7         6        14
1      MBC        25         7         7        11
2      KBS        26         8         3        15
3     JTBC        22        10         4         8


In [40]:
df_info.to_csv(f"result_per_category.csv", encoding='utf-8-sig', index=False)